# SNP Table Workflow

This notebook contains the workflow completed to replicate Table 2. Minor nucleotide variant identified from WHUO1 and WHUO2 genomes

### Workflow Description
__indexing reference__
* purpose: prepare the reference genome for read alignment using BWA 
* the reference genome (`ref`) is indexed by bwa

__read alignment (BWA)__
* purpose: align cleaned sequencing reads from each patient to the reference genome
* each merged fastq file (paired-end reads concatenated) is aligned using bwa mem
* the output is saved as a .sam file (sequence alignment/map format)

__converting sam to bam & sorting (SAMtools)__
* purpose: convert and organize alignment data to be efficient for variant calling
* .sam files are converted to .bam (binary format)
* BAM files are sorted and indexed for random access by variant callers

__variant calling (bcftools)__
* purpose: identify genomic differences (SNPs, indels) between each sample and the reference genome
* `bctools mpileuup` reads alignment data and calculates genotype likelihoods
* `bcftools call` identifies variants based on likelihoods and outputs a .vcf file

__parsing vcf to a table__
* purpose: transform raw variant data into a readable and filterable table for downstream analysis
* each vcf file is parsed using vcfpy
* for each variant, position, reference/alternate allele, coverage, quality, and type are extracted
* data is stored in pandas df
    * strain: patient identifier (WHU01/WHU02)
    * start position: ref genome position of the variant
    * end position: end position (useful for indels)
    * length: length of ref allele
    * change: description of mutation
    * coverage: read depth at the variant site
    * polymorphism type: SNP or indel
    * variant frequency: NA
    * quality: variant confidence score: NA

In [5]:
import subprocess
import os

!pwd

/Users/lilliang/Documents/Spring_2025/EEOB5460/EEOB5460_final_project2025


In [2]:
# function to print all rows of a df (useful when troubleshooting data clean-up steps)
def print_full(x):
    import pandas as pd
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        print(x)

In [3]:
# files
ref = "working_files/viral_ref_genome_output.fasta"
samples = {
    "WHU01": "working_files/wuhan1_merged.fastq",
    "WHU02": "working_files/wuhan2_merged.fastq"
}

In [4]:
# index reference (only once)
subprocess.run(["bwa", "index", ref])

[bwa_index] Pack FASTA... 3.24 sec
[bwa_index] Construct BWT for the packed sequence...
[BWTIncCreate] textLength=1103023696, availableWord=89612688
[BWTIncConstructFromPacked] 10 iterations done. 100000000 characters processed.
[BWTIncConstructFromPacked] 20 iterations done. 200000000 characters processed.
[BWTIncConstructFromPacked] 30 iterations done. 300000000 characters processed.
[BWTIncConstructFromPacked] 40 iterations done. 400000000 characters processed.
[BWTIncConstructFromPacked] 50 iterations done. 496873216 characters processed.
[BWTIncConstructFromPacked] 60 iterations done. 583320176 characters processed.
[BWTIncConstructFromPacked] 70 iterations done. 660150432 characters processed.
[BWTIncConstructFromPacked] 80 iterations done. 728433360 characters processed.
[BWTIncConstructFromPacked] 90 iterations done. 789119392 characters processed.
[BWTIncConstructFromPacked] 100 iterations done. 843053280 characters processed.
[BWTIncConstructFromPacked] 110 iterations done. 8

CompletedProcess(args=['bwa', 'index', 'working_files/viral_ref_genome_output.fasta'], returncode=0)

In [6]:
# align reads to reference, convert to sorted BAM and index 
for name, fastq in samples.items():
    sam = os.path.join("working_files", f"{name}.sam")
    bam = os.path.join("working_files", f"{name}.bam")
    sorted_bam = os.path.join("working_files", f"{name}_sorted.bam")

    subprocess.run(["bwa", "mem", ref, fastq], stdout=open(sam, "w"), check = True)
    subprocess.run(["samtools", "view", "-bS", sam, "-o", bam], check = True)
    subprocess.run(["samtools", "sort", bam, "-o", sorted_bam], check = True)
    subprocess.run(["samtools", "index", sorted_bam], check = True)

[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 71864 sequences (10000209 bp)...
[M::process] read 71648 sequences (10000178 bp)...
[M::mem_process_seqs] Processed 71864 reads in 6.937 CPU sec, 6.921 real sec
[M::process] read 71950 sequences (10000031 bp)...
[M::mem_process_seqs] Processed 71648 reads in 5.875 CPU sec, 5.774 real sec
[M::process] read 71656 sequences (10000220 bp)...
[M::mem_process_seqs] Processed 71950 reads in 5.695 CPU sec, 5.592 real sec
[M::process] read 71774 sequences (10000249 bp)...
[M::mem_process_seqs] Processed 71656 reads in 5.490 CPU sec, 5.447 real sec
[M::process] read 71960 sequences (10000285 bp)...
[M::mem_process_seqs] Processed 71774 reads in 6.257 CPU sec, 6.150 real sec
[M::process] read 71732 sequences (10000281 bp)...
[M::mem_process_seqs] Processed 71960 reads in 6.888 CPU sec, 6.781 real sec
[M::process] read 72228 sequences (10000021 bp)...
[M::mem_process_seqs] Processed 71732 reads in 7.412 CPU sec, 7.306 real sec
[M::pr

In [7]:
# variant calling: create VCFs
for name in samples.keys():
    sorted_bam = os.path.join("working_files", f"{name}_sorted.bam")
    raw_bcf = os.path.join("working_files", f"{name}_raw.bcf")
    vcf_file = os.path.join("working_files", f"{name}.vcf")

    with open(raw_bcf, "wb") as out_bcf:
        subprocess.run(["bcftools", "mpileup", "-f", ref, sorted_bam, "-Ou"], stdout=out_bcf)
    subprocess.run(["bcftools", "call", "-mv", "-Ov", "-o", vcf_file, raw_bcf])

[mpileup] 1 samples in 1 input files
[mpileup] maximum number of reads per input file set to -d 250
Note: none of --samples-file, --ploidy or --ploidy-file given, assuming all sites are diploid
[mpileup] 1 samples in 1 input files
[mpileup] maximum number of reads per input file set to -d 250
Note: none of --samples-file, --ploidy or --ploidy-file given, assuming all sites are diploid


In [8]:
import pandas as pd
import vcfpy as vcf 

def vcf_to_dataframe(vcf_file, strain):
    vcf_reader = vcf.Reader(open(vcf_file, 'r'))
    rows = []

    for record in vcf_reader:
        # safely extract ALT allele(s)
        alt_alleles = record.ALT
        try:
            alt = str(alt_alleles[0]) if alt_alleles else None
        except Exception:
            alt = None

        # determine length safely
        ref_len = len(record.REF) if record.REF else 0
        end_pos = record.POS + ref_len - 1 if record.POS and ref_len else record.POS

        # handle AF (allele frequency) values
        af_raw = record.INFO.get("AF", None)
        if isinstance(af_raw, list) and af_raw:
            af_percent = af_raw[0] * 100
        elif isinstance(af_raw, float):
            af_percent = af_raw * 100
        else:
            af_percent = None

        # determine polymorphism type
        polymorphism_type = "SNP" if ref_len == 1 and alt and len(str(alt)) == 1 else "Indel"

        rows.append({
            "Strain": strain,
            "Start Position": record.POS,
            "End Position": end_pos,
            "Length": ref_len,
            "Change": f"{record.REF} → {alt}" if alt else None,
            "Coverage": record.INFO.get("DP", None),
            "Polymorphism Type": polymorphism_type,
            "Variant Frequency (%)": af_percent,
            "Quality": record.QUAL
        })

    return pd.DataFrame(rows)

In [9]:
df1 = vcf_to_dataframe("working_files/WHU01.vcf", "WHU01")
df2 = vcf_to_dataframe("working_files/WHU02.vcf", "WHU02")

variants_df = pd.concat([df1, df2]).sort_values(by=["Strain", "Start Position"])
variants_df.reset_index(drop=True, inplace=True)

In [10]:
print_full(variants_df)

    Strain  Start Position  End Position  Length  \
0    WHU01               2             2       1   
1    WHU01              23            23       1   
2    WHU01              34            34       1   
3    WHU01              40            40       1   
4    WHU01              51            51       1   
5    WHU01              59            59       1   
6    WHU01              66            66       1   
7    WHU01              67            67       1   
8    WHU01             103           103       1   
9    WHU01             113           113       1   
10   WHU01             117           117       1   
11   WHU01             118           118       1   
12   WHU01             127           127       1   
13   WHU01             134           134       1   
14   WHU01             137           137       1   
15   WHU01             139           139       1   
16   WHU01             169           169       1   
17   WHU01             178           178       1   
18   WHU01  

In [11]:
html_table = variants_df.to_html(index=False)
output_path = "output_files/figures/variants.html"
with open(output_path, "w") as f:
    f.write(html_table)